# 🧭 QAOA para o Problema do Caixeiro Viajante (TSP)

## Implementação Didática com Execução no IBM Quantum

Este notebook implementa o TSP usando QAOA seguindo a formulação matemática:

1. **Variáveis de decisão**: $x_{i,t} \in \{0,1\}$
2. **Formulação QUBO**: $H_{QUBO} = H_{dist} + H_{p1} + H_{p2}$
3. **Mapeamento quântico**: $x_{i,t} \rightarrow \frac{I - \hat{Z}_{i,t}}{2}$
4. **QAOA**: Operadores $U_C(\gamma)$ e $U_M(\beta)$

### Comparações Realizadas
- **Brute Force** (clássico ótimo)
- **QAOA Simulador** (Aer - sem ruído)
- **QAOA IBM Quantum** (hardware real - com ruído)

---
## 📦 Instalação de Dependências (Google Colab)

In [ ]:
# Instalar pacotes necessários no Colab
!pip install qiskit>=1.0 -q
!pip install qiskit-aer -q
!pip install qiskit-ibm-runtime -q

print("✅ Pacotes instalados com sucesso!")

In [ ]:
# Imports
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Session
import numpy as np
from scipy.optimize import minimize
from itertools import permutations
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# Simulador local
sim_local = AerSimulator()

print("✅ Bibliotecas importadas com sucesso!")

---
## 🔑 Configuração do IBM Quantum Cloud

In [ ]:
# =============================================================================
# CREDENCIAIS IBM QUANTUM (IBM CLOUD)
# =============================================================================

# Salvar credenciais (execute apenas UMA VEZ)
QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token="sss",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b58cd9ceef164e4586608dca729daacc:6117674a-f3e7-4bc3-8062-30e14b62d3be::",
    overwrite=True
)

print("✅ Credenciais salvas com sucesso!")

In [ ]:
# =============================================================================
# CONECTAR AO IBM QUANTUM
# =============================================================================

service = QiskitRuntimeService(
    channel="ibm_cloud", 
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b58cd9ceef164e4586608dca729daacc:6117674a-f3e7-4bc3-8062-30e14b62d3be::"
)

print("✅ Conectado ao IBM Quantum!")

# Listar backends disponíveis
print("\n📡 Backends disponíveis na sua conta:")
print("-" * 60)
backends = service.backends()
for b in backends:
    status = "🟢" if b.status().operational else "🔴"
    print(f"   {status} {b.name}: {b.num_qubits} qubits")

In [ ]:
# =============================================================================
# SELECIONAR BACKEND IBM QUANTUM
# =============================================================================

# Seleciona ibm_fez (156 qubits, Heron r2, menor tempo de espera)
backend_ibm = service.backend("ibm_fez")

# Alternativas:
# backend_ibm = service.backend("ibm_marrakesh")  # 156 qubits
# backend_ibm = service.backend("ibm_torino")     # 133 qubits

print(f"\n🎯 Backend selecionado: {backend_ibm.name}")
print(f"   Qubits disponíveis: {backend_ibm.num_qubits}")
print(f"   Status: {'Operacional ✅' if backend_ibm.status().operational else 'Offline ❌'}")

---
## 📊 Definição do Problema TSP

### Variáveis de Decisão $x_{i,t}$

$$x_{i,t} = \begin{cases} 1 & \text{se a cidade } i \text{ é visitada no passo } t \\ 0 & \text{caso contrário} \end{cases}$$

Para $n$ cidades, temos $n^2$ variáveis binárias, ou seja, **$n^2$ qubits**.

In [ ]:
# Matrizes de distâncias
graphs = {
    3: np.array([
        [0, 10, 15],
        [10, 0, 20],
        [15, 20, 0]
    ], dtype=float),
    
    4: np.array([
        [0, 1, 50, 50],
        [1, 0, 2, 50],
        [50, 2, 0, 3],
        [50, 50, 3, 0]
    ], dtype=float),
    
    5: np.array([
        [0, 2, 9, 10, 7],
        [1, 0, 6, 4, 3],
        [15, 7, 0, 8, 3],
        [6, 3, 12, 0, 11],
        [9, 7, 5, 6, 0]
    ], dtype=float),
    
    6: np.array([
        [0, 3, 6, 7, 8, 9],
        [3, 0, 5, 6, 7, 8],
        [6, 5, 0, 4, 5, 6],
        [7, 6, 4, 0, 3, 4],
        [8, 7, 5, 3, 0, 2],
        [9, 8, 6, 4, 2, 0]
    ], dtype=float)
}

# Matrizes para IBM Quantum (apenas 3 e 4 cidades - viáveis em hardware)
graphs_ibm = {k: v for k, v in graphs.items() if k <= 4}

for n, D in graphs.items():
    print(f"\n📍 Matriz D ({n} cidades) - {n**2} qubits:")
    print(D.astype(int))

In [ ]:
# Mapeamento (cidade, tempo) → índice do qubit

def qubit_index(cidade, tempo, n):
    """
    Mapeia a variável x_{i,t} para o índice do qubit.
    x_{i,t} → qubit[i * n + t]
    """
    return cidade * n + tempo


# Visualização do mapeamento para n=3
print("🔢 Mapeamento de variáveis x_{i,t} para qubits (n=3):")
print("="*50)
print(f"{'Variável':<12} {'Qubit':<8} {'Significado'}")
print("-"*50)
for i in range(3):
    for t in range(3):
        q = qubit_index(i, t, 3)
        print(f"x_{{{i},{t}}}        q[{q}]     Cidade {i} no tempo {t}")

---
## 📐 Construção do Hamiltoniano de Custo

### Formulação QUBO

$$H_{QUBO} = \underbrace{\sum_{i,j=0}^{n-1} \sum_{t=0}^{n-1} d_{ij} \, x_{i,t} \, x_{j,(t+1) \mod n}}_{H_{dist}} + \underbrace{A \sum_{i=0}^{n-1} \left( \sum_{t=0}^{n-1} x_{i,t} - 1 \right)^2}_{H_{p1}} + \underbrace{A \sum_{t=0}^{n-1} \left( \sum_{i=0}^{n-1} x_{i,t} - 1 \right)^2}_{H_{p2}}$$

### Mapeamento para Operadores de Pauli

$$x_{i,t} \longrightarrow \hat{n}_{i,t} = \frac{I - \hat{Z}_{i,t}}{2}$$

In [ ]:
def construir_hamiltoniano_tsp(D, penalty_multiplier=2.0):
    """
    Constrói os coeficientes do Hamiltoniano de custo para o TSP.
    
    H_QUBO = H_dist + A*H_p1 + A*H_p2
    Mapeamento: x_{i,t} → (I - Z_{i,t})/2
    
    Retorna:
    --------
    h : dict - Coeficientes dos termos Z_i (campo local)
    J : dict - Coeficientes dos termos Z_i Z_j (interação)
    A : float - Fator de penalidade
    """
    n = len(D)
    A = penalty_multiplier * np.max(D) * n
    
    h = {q: 0.0 for q in range(n * n)}
    J = {}
    
    # H_dist: custo das distâncias
    for i in range(n):
        for j in range(n):
            if i != j:
                d_ij = D[i, j]
                for t in range(n):
                    t_next = (t + 1) % n
                    q_a = qubit_index(i, t, n)
                    q_b = qubit_index(j, t_next, n)
                    h[q_a] -= d_ij / 4
                    h[q_b] -= d_ij / 4
                    key = (min(q_a, q_b), max(q_a, q_b))
                    J[key] = J.get(key, 0) + d_ij / 4
    
    # H_p1: cada cidade visitada exatamente uma vez
    for i in range(n):
        for t in range(n):
            q = qubit_index(i, t, n)
            h[q] += A / 2
            for t2 in range(t + 1, n):
                q2 = qubit_index(i, t2, n)
                h[q] -= A / 2
                h[q2] -= A / 2
                key = (min(q, q2), max(q, q2))
                J[key] = J.get(key, 0) + A / 2
    
    # H_p2: cada tempo tem exatamente uma cidade
    for t in range(n):
        for i in range(n):
            q = qubit_index(i, t, n)
            h[q] += A / 2
            for i2 in range(i + 1, n):
                q2 = qubit_index(i2, t, n)
                h[q] -= A / 2
                h[q2] -= A / 2
                key = (min(q, q2), max(q, q2))
                J[key] = J.get(key, 0) + A / 2
    
    return h, J, A


# Demonstração
h_demo, J_demo, A_demo = construir_hamiltoniano_tsp(graphs[3])
print(f"\n📐 Hamiltoniano para n=3:")
print(f"   Penalidade A = {A_demo}")
print(f"   Termos Z_i: {len([v for v in h_demo.values() if abs(v) > 1e-10])}")
print(f"   Termos Z_i Z_j: {len(J_demo)}")

---
## 🔄 Camada (Layer) do QAOA

**Operador de Custo:**
$$U_C(\gamma) = e^{-i \gamma \hat{H}_C}$$

**Operador de Mistura (Mixer):**
$$U_M(\beta) = e^{-i \beta \hat{H}_M}, \quad \hat{H}_M = \sum_{k=1}^{n^2} \hat{X}_k$$

**Implementação:**
- $e^{-i\gamma h_k Z_k}$ → **RZ(2γh_k)**
- $e^{-i\gamma J_{ij} Z_i Z_j}$ → **CNOT · RZ · CNOT**
- $e^{-i\beta X_k}$ → **RX(2β)**

In [ ]:
def qaoa_layer(qc, h, J, gamma, beta):
    """
    Implementa uma camada do QAOA.
    
    U_C(γ) = exp(-iγ Ĥ_C)
    U_M(β) = exp(-iβ Σ X_k)
    """
    # Operador de Custo U_C(γ)
    for qubit, coef in h.items():
        if abs(coef) > 1e-10:
            qc.rz(2 * gamma * coef, qubit)
    
    for (q_i, q_j), coef in J.items():
        if abs(coef) > 1e-10:
            qc.cx(q_i, q_j)
            qc.rz(2 * gamma * coef, q_j)
            qc.cx(q_i, q_j)
    
    # Operador Mixer U_M(β)
    for q in range(qc.num_qubits):
        qc.rx(2 * beta, q)

---
## 🔌 Circuito Completo do QAOA

$$|\psi(\vec{\gamma}, \vec{\beta})\rangle = \prod_{l=1}^{p} U_M(\beta_l) U_C(\gamma_l) |+\rangle^{\otimes n^2}$$

In [ ]:
def qaoa_circuit(h, J, num_qubits, gammas, betas):
    """
    Constrói o circuito QAOA completo.
    
    |ψ(γ,β)⟩ = Π_{l=1}^{p} U_M(β_l) U_C(γ_l) |+⟩^{⊗n²}
    """
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))  # Estado inicial |+⟩
    
    for gamma, beta in zip(gammas, betas):
        qaoa_layer(qc, h, J, gamma, beta)
    
    return qc


# Visualizar circuito para n=3, p=1
qc_exemplo = qaoa_circuit(h_demo, J_demo, 9, [0.5], [0.5])
print(f"📊 Circuito QAOA (n=3, p=1):")
print(f"   Qubits: {qc_exemplo.num_qubits}")
print(f"   Profundidade: {qc_exemplo.depth()}")
print(f"   Portas: {dict(qc_exemplo.count_ops())}")

qc_exemplo.draw('mpl', fold=80)

---
## 💰 Funções de Custo e Decodificação

### Soluções Válidas
Uma bitstring é válida se satisfaz:
1. $\sum_{t} x_{i,t} = 1$ (cada cidade uma vez)
2. $\sum_{i} x_{i,t} = 1$ (cada tempo uma cidade)

In [ ]:
def decodificar_bitstring(bitstring, n):
    """Decodifica bitstring em rota TSP."""
    x = np.zeros((n, n), dtype=int)
    for idx, bit in enumerate(bitstring):
        cidade = idx // n
        tempo = idx % n
        x[cidade, tempo] = int(bit)
    
    # Verifica restrições
    for i in range(n):
        if np.sum(x[i, :]) != 1:
            return None, False
    for t in range(n):
        if np.sum(x[:, t]) != 1:
            return None, False
    
    # Extrai rota
    rota = []
    for t in range(n):
        for i in range(n):
            if x[i, t] == 1:
                rota.append(i)
                break
    rota.append(rota[0])
    return tuple(rota), True


def tsp_cost(bitstring, D):
    """Calcula custo de uma solução TSP."""
    n = len(D)
    rota, valida = decodificar_bitstring(bitstring, n)
    if not valida:
        return float('inf'), None, False
    custo = sum(D[rota[t], rota[t + 1]] for t in range(n))
    return custo, rota, True


def processar_counts(counts, D):
    """Processa contagens de medição."""
    n = len(D)
    total_shots = sum(counts.values())
    penalidade = np.max(D) * n * 10
    
    exp_cost = 0
    n_validas = 0
    melhor_custo = float('inf')
    melhor_rota = None
    
    for bitstring, count in counts.items():
        bs = bitstring.replace(" ", "")[::-1]
        custo, rota, valida = tsp_cost(bs, D)
        
        if valida:
            n_validas += count
            exp_cost += count * custo
            if custo < melhor_custo:
                melhor_custo = custo
                melhor_rota = rota
        else:
            exp_cost += count * penalidade
    
    return {
        'exp_cost': exp_cost / total_shots,
        'frac_validas': n_validas / total_shots,
        'melhor_rota': melhor_rota,
        'melhor_custo': melhor_custo
    }


def brute_force_tsp(D):
    """Resolve TSP por força bruta (solução ótima)."""
    n = len(D)
    melhor_rota = None
    melhor_custo = float('inf')
    for perm in permutations(range(1, n)):
        rota = (0,) + perm + (0,)
        custo = sum(D[rota[i], rota[i+1]] for i in range(n))
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_rota = rota
    return melhor_rota, melhor_custo

---
## 🎯 Ciclo Híbrido: Otimização com COBYLA

O QAOA opera em ciclo híbrido clássico-quântico:
1. **Quântico**: Prepara $|\psi(\gamma, \beta)\rangle$ e mede
2. **Clássico**: Calcula $\langle H_C \rangle$ e ajusta parâmetros

In [ ]:
def objective_simulador(params, h, J, D, num_qubits, p, shots=2048):
    """Função objetivo usando simulador local."""
    gammas = params[:p]
    betas = params[p:]
    
    qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
    qc.measure_all()
    
    tqc = transpile(qc, sim_local)
    result = sim_local.run(tqc, shots=shots).result()
    counts = result.get_counts()
    
    res = processar_counts(counts, D)
    return res['exp_cost']


def otimizar_parametros(D, p=2, maxiter=200):
    """Otimiza parâmetros γ e β usando simulador local."""
    n = len(D)
    num_qubits = n ** 2
    h, J, A = construir_hamiltoniano_tsp(D)
    
    init_params = np.random.uniform(0, np.pi, 2 * p)
    
    resultado = minimize(
        objective_simulador,
        init_params,
        args=(h, J, D, num_qubits, p),
        method="COBYLA",
        options={'maxiter': maxiter}
    )
    
    return resultado.x[:p], resultado.x[p:], h, J, A

---
## 🚀 Execução Completa: Simulador + IBM Quantum

### Parâmetros do Experimento

In [ ]:
# Parâmetros do experimento
PARAMS = {
    'p': 3,              # Camadas QAOA
    'shots': 8192,       # Medições
    'maxiter': 300,      # Iterações COBYLA
    'penalty': 2.0       # Multiplicador penalidade
}

print("⚙️ Parâmetros do Experimento:")
for k, v in PARAMS.items():
    print(f"   {k}: {v}")

In [ ]:
def executar_comparacao_completa(graphs_sim, graphs_ibm, backend_ibm, params):
    """
    Executa comparação completa:
    - Brute Force (todos)
    - QAOA Simulador (todos)
    - QAOA IBM Quantum (3 e 4 cidades apenas)
    """
    resultados = []
    parametros_otimizados = {}
    
    # =========================================================================
    # FASE 1: BRUTE FORCE + QAOA SIMULADOR (todas as instâncias)
    # =========================================================================
    print("\n" + "="*70)
    print("📊 FASE 1: BRUTE FORCE + QAOA SIMULADOR")
    print("="*70)
    
    for n_cidades, D in graphs_sim.items():
        print(f"\n{'='*60}")
        print(f"📍 PROCESSANDO: {n_cidades} CIDADES ({n_cidades**2} qubits)")
        print(f"{'='*60}")
        
        n = len(D)
        num_qubits = n ** 2
        
        # BRUTE FORCE
        print(f"\n[1/2] 🔍 Brute Force...")
        inicio_bf = time.time()
        rota_bf, custo_bf = brute_force_tsp(D)
        tempo_bf = time.time() - inicio_bf
        print(f"      ✅ Rota: {rota_bf}, Custo: {custo_bf}, Tempo: {tempo_bf:.6f}s")
        
        # QAOA SIMULADOR
        print(f"\n[2/2] ⚛️ QAOA Simulador (p={params['p']})...")
        inicio_sim = time.time()
        
        gammas, betas, h, J, A = otimizar_parametros(D, p=params['p'], maxiter=params['maxiter'])
        
        # Execução final
        qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
        qc.measure_all()
        tqc = transpile(qc, sim_local)
        result = sim_local.run(tqc, shots=params['shots']).result()
        counts = result.get_counts()
        
        tempo_sim = time.time() - inicio_sim
        res_sim = processar_counts(counts, D)
        
        # Calcular gap
        if res_sim['melhor_rota'] is not None:
            gap_sim = ((res_sim['melhor_custo'] - custo_bf) / custo_bf) * 100
        else:
            gap_sim = float('inf')
        
        print(f"      ✅ Rota: {res_sim['melhor_rota']}")
        print(f"      ✅ Custo: {res_sim['melhor_custo']}")
        print(f"      ✅ Válidas: {100*res_sim['frac_validas']:.2f}%")
        print(f"      ✅ Gap: {gap_sim:.2f}%")
        print(f"      ✅ Tempo: {tempo_sim:.2f}s")
        
        # Salvar parâmetros para IBM
        if n_cidades in graphs_ibm:
            parametros_otimizados[n_cidades] = {
                'gammas': gammas, 'betas': betas, 'h': h, 'J': J, 'D': D
            }
        
        # Armazenar resultado
        resultados.append({
            'Cidades': n_cidades,
            'Qubits': num_qubits,
            'Rota BF': str(rota_bf),
            'Custo BF': custo_bf,
            'Tempo BF (s)': tempo_bf,
            'Rota Sim': str(res_sim['melhor_rota']),
            'Custo Sim': res_sim['melhor_custo'] if res_sim['melhor_custo'] != float('inf') else 'N/A',
            'Gap Sim (%)': gap_sim if gap_sim != float('inf') else 'N/A',
            'Válidas Sim (%)': res_sim['frac_validas'] * 100,
            'Tempo Sim (s)': tempo_sim,
            'Rota IBM': None,
            'Custo IBM': None,
            'Gap IBM (%)': None,
            'Válidas IBM (%)': None,
            'Tempo IBM (s)': None
        })
    
    # =========================================================================
    # FASE 2: QAOA IBM QUANTUM (apenas 3 e 4 cidades)
    # =========================================================================
    print("\n" + "="*70)
    print(f"☁️ FASE 2: QAOA IBM QUANTUM ({backend_ibm.name})")
    print("="*70)
    
    with Session(service=service, backend=backend_ibm) as session:
        sampler = Sampler(session=session)
        
        for n_cidades, params_opt in parametros_otimizados.items():
            print(f"\n📍 IBM Quantum: {n_cidades} cidades ({n_cidades**2} qubits)...")
            
            D = params_opt['D']
            h, J = params_opt['h'], params_opt['J']
            gammas, betas = params_opt['gammas'], params_opt['betas']
            num_qubits = n_cidades ** 2
            
            # Construir circuito
            qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
            qc.measure_all()
            
            # Transpilar
            print(f"   🔧 Transpilando...")
            qc_transpiled = transpile(qc, backend=backend_ibm, optimization_level=3)
            print(f"      Profundidade: {qc_transpiled.depth()}")
            
            # Executar
            print(f"   🚀 Enviando job...")
            inicio_ibm = time.time()
            
            job = sampler.run([qc_transpiled], shots=params['shots'])
            print(f"      Job ID: {job.job_id()}")
            print(f"   ⏳ Aguardando...")
            
            result = job.result()
            tempo_ibm = time.time() - inicio_ibm
            
            # Processar
            pub_result = result[0]
            counts = pub_result.data.meas.get_counts()
            res_ibm = processar_counts(counts, D)
            
            rota_bf, custo_bf = brute_force_tsp(D)
            if res_ibm['melhor_rota'] is not None:
                gap_ibm = ((res_ibm['melhor_custo'] - custo_bf) / custo_bf) * 100
            else:
                gap_ibm = float('inf')
            
            print(f"   ✅ Rota: {res_ibm['melhor_rota']}")
            print(f"   ✅ Custo: {res_ibm['melhor_custo']}")
            print(f"   ✅ Válidas: {100*res_ibm['frac_validas']:.2f}%")
            print(f"   ✅ Gap: {gap_ibm:.2f}%")
            print(f"   ✅ Tempo: {tempo_ibm:.2f}s")
            
            # Atualizar resultado
            for r in resultados:
                if r['Cidades'] == n_cidades:
                    r['Rota IBM'] = str(res_ibm['melhor_rota'])
                    r['Custo IBM'] = res_ibm['melhor_custo'] if res_ibm['melhor_custo'] != float('inf') else 'N/A'
                    r['Gap IBM (%)'] = gap_ibm if gap_ibm != float('inf') else 'N/A'
                    r['Válidas IBM (%)'] = res_ibm['frac_validas'] * 100
                    r['Tempo IBM (s)'] = tempo_ibm
                    break
    
    return pd.DataFrame(resultados)

In [ ]:
# =============================================================================
# 🚀 EXECUTAR COMPARAÇÃO COMPLETA
# =============================================================================

print("="*70)
print("🚀 INICIANDO COMPARAÇÃO: BRUTE FORCE vs SIMULADOR vs IBM QUANTUM")
print("="*70)

df_resultados = executar_comparacao_completa(graphs, graphs_ibm, backend_ibm, PARAMS)

print(f"\n{'='*70}")
print("✅ PROCESSAMENTO CONCLUÍDO!")
print("="*70)

---
## 📊 Resultados e Análise

In [ ]:
# Tabela completa
print("\n" + "="*70)
print("📊 TABELA COMPLETA DE RESULTADOS")
print("="*70 + "\n")
display(df_resultados)

In [ ]:
# Tabela resumida (conforme enunciado)
print("\n" + "="*70)
print("📋 RESUMO: TEMPO, CUSTO E DISTÂNCIA RELATIVA AO ÓTIMO")
print("="*70 + "\n")

df_resumo = df_resultados[[
    'Cidades', 'Qubits',
    'Tempo BF (s)', 'Tempo Sim (s)', 'Tempo IBM (s)',
    'Custo BF', 'Custo Sim', 'Custo IBM',
    'Gap Sim (%)', 'Gap IBM (%)',
    'Válidas Sim (%)', 'Válidas IBM (%)'
]].copy()

df_resumo.columns = [
    'Cidades', 'Qubits',
    'Tempo Clássico', 'Tempo Simulador', 'Tempo IBM',
    'Custo Ótimo', 'Custo Simulador', 'Custo IBM',
    'Gap Sim (%)', 'Gap IBM (%)',
    'Válidas Sim (%)', 'Válidas IBM (%)'
]

display(df_resumo)

In [ ]:
# =============================================================================
# GRÁFICOS DE ANÁLISE
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

x = df_resultados['Cidades'].values
width = 0.25

# Gráfico 1: Tempos de Execução
ax1 = axes[0, 0]
tempos_ibm = [t if t is not None else 0 for t in df_resultados['Tempo IBM (s)']]
ax1.bar(x - width, df_resultados['Tempo BF (s)'], width, label='Brute Force', color='green')
ax1.bar(x, df_resultados['Tempo Sim (s)'], width, label='Simulador', color='steelblue')
ax1.bar(x + width, tempos_ibm, width, label='IBM Quantum', color='purple')
ax1.set_xlabel('Número de Cidades')
ax1.set_ylabel('Tempo (s)')
ax1.set_title('Tempo de Execução')
ax1.legend()
ax1.set_xticks(x)
ax1.set_yscale('log')

# Gráfico 2: Custos
ax2 = axes[0, 1]
custos_sim = [c if c != 'N/A' else 0 for c in df_resultados['Custo Sim']]
custos_ibm = [c if c not in ['N/A', None] else 0 for c in df_resultados['Custo IBM']]
ax2.bar(x - width, df_resultados['Custo BF'], width, label='Brute Force', color='green')
ax2.bar(x, custos_sim, width, label='Simulador', color='steelblue')
ax2.bar(x + width, custos_ibm, width, label='IBM Quantum', color='purple')
ax2.set_xlabel('Número de Cidades')
ax2.set_ylabel('Custo da Rota')
ax2.set_title('Custo: Ótimo vs QAOA')
ax2.legend()
ax2.set_xticks(x)

# Gráfico 3: Gap Relativo
ax3 = axes[1, 0]
gaps_sim = [g if g not in ['N/A', float('inf')] else 100 for g in df_resultados['Gap Sim (%)']]
gaps_ibm = [g if g not in ['N/A', None, float('inf')] else 0 for g in df_resultados['Gap IBM (%)']]
x_sim = x
x_ibm = x[:len(graphs_ibm)]
ax3.bar(x_sim - width/2, gaps_sim, width, label='Simulador', color='steelblue')
ax3.bar(x_ibm + width/2, gaps_ibm[:len(graphs_ibm)], width, label='IBM Quantum', color='purple')
ax3.set_xlabel('Número de Cidades')
ax3.set_ylabel('Gap (%)')
ax3.set_title('Distância Relativa ao Ótimo')
ax3.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='Ótimo')
ax3.legend()
ax3.set_xticks(x)

# Gráfico 4: Soluções Válidas
ax4 = axes[1, 1]
validas_ibm = [v if v is not None else 0 for v in df_resultados['Válidas IBM (%)']]
ax4.bar(x - width/2, df_resultados['Válidas Sim (%)'], width, label='Simulador', color='steelblue')
ax4.bar(x + width/2, validas_ibm, width, label='IBM Quantum', color='purple')
ax4.set_xlabel('Número de Cidades')
ax4.set_ylabel('Soluções Válidas (%)')
ax4.set_title('Porcentagem de Soluções Válidas')
ax4.legend()
ax4.set_xticks(x)

plt.tight_layout()
plt.savefig('comparacao_completa_tsp_qaoa.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📁 Gráfico salvo como 'comparacao_completa_tsp_qaoa.png'")

In [ ]:
# =============================================================================
# ANÁLISE DE DESEMPENHO E LIMITAÇÕES
# =============================================================================

print("\n" + "="*70)
print("📈 ANÁLISE DE DESEMPENHO E LIMITAÇÕES")
print("="*70)

print("\n🔹 BRUTE FORCE (Clássico):")
print("-" * 50)
print("   • Sempre encontra a solução ÓTIMA")
print("   • Complexidade: O(n!) - cresce fatorialmente")
print(f"   • Tempo n=3: {df_resultados[df_resultados['Cidades']==3]['Tempo BF (s)'].values[0]:.6f}s")
print(f"   • Tempo n=6: {df_resultados[df_resultados['Cidades']==6]['Tempo BF (s)'].values[0]:.6f}s")

print("\n🔹 QAOA SIMULADOR (sem ruído):")
print("-" * 50)
gaps_sim_validos = [g for g in df_resultados['Gap Sim (%)'] if g not in ['N/A', float('inf')]]
print(f"   • Gap médio: {np.mean(gaps_sim_validos):.2f}%")
print(f"   • Média soluções válidas: {df_resultados['Válidas Sim (%)'].mean():.2f}%")
print(f"   • Representa limite teórico do QAOA")

print("\n🔹 QAOA IBM QUANTUM (hardware real):")
print("-" * 50)
df_ibm_valid = df_resultados[df_resultados['Custo IBM'].notna()]
if len(df_ibm_valid) > 0:
    gaps_ibm_validos = [g for g in df_ibm_valid['Gap IBM (%)'] if g not in ['N/A', None, float('inf')]]
    if gaps_ibm_validos:
        print(f"   • Gap médio: {np.mean(gaps_ibm_validos):.2f}%")
    validas_ibm_list = [v for v in df_ibm_valid['Válidas IBM (%)'] if v is not None]
    if validas_ibm_list:
        print(f"   • Média soluções válidas: {np.mean(validas_ibm_list):.2f}%")
    print(f"   • Backend utilizado: {backend_ibm.name}")
    print(f"   • Afetado por ruído, decoerência e erros de porta")

print("\n🔹 IMPACTO DO RUÍDO (Simulador vs IBM):")
print("-" * 50)
for n in graphs_ibm.keys():
    row = df_resultados[df_resultados['Cidades'] == n].iloc[0]
    if row['Válidas IBM (%)'] is not None:
        diff = row['Válidas Sim (%)'] - row['Válidas IBM (%)']
        print(f"   • n={n}: Redução de {diff:.2f}% em soluções válidas")

print("\n🔹 LIMITAÇÕES OBSERVADAS:")
print("-" * 50)
print("   • QAOA é aproximado (não garante ótimo)")
print("   • Qubits crescem com n² (escalabilidade limitada)")
print("   • Hardware real degrada resultados vs simulador")
print("   • n≥5: 25+ qubits, circuito muito profundo")
print("   • IBM Quantum: viável apenas para n≤4 atualmente")

print("\n🔹 CONCLUSÕES:")
print("-" * 50)
print("   • Para n pequeno (≤4): QAOA funciona em hardware real")
print("   • Simulador é útil para desenvolvimento e teste")
print("   • Brute Force é superior para n≤10 em termos de qualidade")
print("   • Potencial quântico: problemas maiores no futuro")

In [ ]:
# Exportar resultados
df_resultados.to_csv('resultados_tsp_completo.csv', index=False)
print("\n📁 Resultados exportados para 'resultados_tsp_completo.csv'")

---
## 📝 Resumo: Correspondência Teoria ↔ Código

| Conceito Teórico | Implementação |
|-----------------|---------------|
| Variável $x_{i,t}$ | `qubit_index(cidade, tempo, n)` |
| Penalidade $A$ | `penalty_multiplier * max(D) * n` |
| $H_{dist}$ | Loop distâncias em `construir_hamiltoniano_tsp` |
| $H_{p1}, H_{p2}$ | Penalidades por cidade/tempo |
| $x \to (I-Z)/2$ | Coeficientes `h` e `J` |
| $U_C(\gamma)$ | `RZ` + `CNOT-RZ-CNOT` em `qaoa_layer` |
| $U_M(\beta)$ | `RX(2β)` em `qaoa_layer` |
| Estado inicial | `qc.h(range(num_qubits))` |
| Camadas $p$ | `for gamma, beta in zip(gammas, betas)` |
| Otimizador | `minimize(..., method='COBYLA')` |